## Начало




#### Импорт и установка необходимых зависимостей

In [ ]:
# Install PyTorch with CUDA
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# Install additional dependencies
!pip install datasets jupyter matplotlib pandas pillow timm torcheval torchtnt tqdm
# Install older pyzmq and jupyter client versions
!pip install --upgrade "jupyter_client<8" "pyzmq<25"
# Install utility packages
!pip install cjm_pandas_utils cjm_pil_utils cjm_pytorch_utils

Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
# Import Python Standard Library dependencies
from copy import copy
from time import perf_counter
import datetime
from glob import glob
import json
import math
import multiprocessing
import os
from pathlib import Path
import random

# Import utility functions
from cjm_pandas_utils.core import markdown_to_pandas
from cjm_pil_utils.core import resize_img
from cjm_pytorch_utils.core import set_seed, pil_to_tensor, tensor_to_pil, get_torch_device, denorm_img_tensor

# Import HuggingFace Datasets dependencies
from datasets import load_dataset

# Import matplotlib for creating plots
import matplotlib.pyplot as plt

# Import numpy
import numpy as np

# Import pandas module for data manipulation
import pandas as pd

# Set options for Pandas DataFrame display
pd.set_option('max_colwidth', None)  # Do not truncate the contents of cells in the DataFrame
pd.set_option('display.max_rows', None)  # Display all rows in the DataFrame
pd.set_option('display.max_columns', None)  # Display all columns in the DataFrame

# Import PIL for image manipulation
from PIL import Image

# Import timm library
import timm

# Import PyTorch dependencies
import torch
import torch.nn as nn
from torch.amp import autocast
from torch.cuda.amp import GradScaler
from torchvision import transforms
import torchvision.transforms.functional as TF
from torch.utils.data import Dataset, DataLoader
from torchtnt.utils import get_module_summary
from torcheval.metrics import MulticlassAccuracy

# Import tqdm for progress bar
from tqdm.auto import tqdm

# Import DatasetDict from datasets
from datasets import DatasetDict

# Import drive from googlecolab
from google.colab import drive

#### Setting Up

In [ ]:
# seed for debugging
seed = 1234
set_seed(seed)

device = get_torch_device()
print(f'Device: {device}')
dtype = torch.float32

project_name = "image-classifier"
project_dir = Path(f"/content/{project_name}")
project_dir.mkdir(parents=True, exist_ok=True)

dataset_dir = Path("/content/datasets/")
dataset_dir.mkdir(parents=True, exist_ok=True)

models_weight = {}
models_weight['cifar100'] = 'swin_base_patch4_window7_224.ms_in1k--cifar100.pth'
models_weight['food101'] = 'swin_base_patch4_window7_224.ms_in1k--food101.pth'
models_weight['zh-plus/tiny-imagenet'] = 'swin_base_patch4_window7_224.ms_in1k--food101.pth'

Device: cpu


In [ ]:
drive_dir = Path('/content/drive/MyDrive')
models_dir = Path(f'{drive_dir}/models_data')
if not os.path.exists(drive_dir):
  mount_dir = '/content/drive'
  drive.mount(mount_dir)


## Подготовка моделей

Здесь объявляется свой класс модели и создается список моделей

#### Класс Model

In [ ]:
class Model:
    def __init__(self, model_name, device, dtype):
        # the model itself
        # it is created using 'timm', when receiving the number of classes from the dataset
        self.model = None
        # model configures
        self.conf = None
        # std and mean
        self.norm_stats = None
        # the size of the image that the model expects to see
        self.image_size = None
        # transform for image from dataset
        self.tfms = None

        # loaders for training/validating/testing
        self.training_data_loader = None
        self.testing_data_loader = None
        self.validating_data_loader = None

        self.device = device
        self.dtype = dtype
        self.model_name = model_name

    # called when the dataset is defined
    def init_model_for_dataset(self, device, dtype, num_classes):
        self.model = timm.create_model(self.model_name, pretrained=True, num_classes=num_classes)
        self.model = self.model.to(device=device, dtype=dtype)
        self.model.device = device

        self.conf = self.model.default_cfg
        self.norm_stats = (self.conf['mean'], self.conf['std'])
        print(self.norm_stats)
        self.image_size = self.conf['input_size'][1:3]
        self.tfms = transforms.Compose([
          transforms.Resize(self.image_size),
          transforms.ToTensor(),
          transforms.Normalize(*self.norm_stats)
        ])

    # need to prevent memory loss during repeated cell launches
    # free memory in cpu/gpu ram
    def delete(self):
      self.clear_loaders()
      if self.model is not None:
        del self.model
        self.model = None

    # need to prevent memory loss during repeated cell launches
    # free memory in cpu/gpu ram
    def clear_loaders(self):
      def clear_loader(loader):
        if loader is None:
          return
        del loader
      clear_loader(self.testing_data_loader)
      clear_loader(self.training_data_loader)
      clear_loader(self.validating_data_loader)


#### Набор моделей, доступных в библиотеке timm

В библиотеке timm доступно 33 модели Swin Transformer. Среди них 19 подходящих моделей.
Однако, в связи с ограниченностью ресурсов и времени, будет рассмотрено всего лишь 2 модели.
При инициализации списка моделей будет использован тот факт, что модели идут последовательно

In [ ]:
pd.DataFrame(timm.list_models('swin*', pretrained=True))

,0
0,swin_base_patch4_window7_224.ms_in1k
1,swin_base_patch4_window7_224.ms_in22k
2,swin_base_patch4_window7_224.ms_in22k_ft_in1k
3,swin_base_patch4_window12_384.ms_in1k
4,swin_base_patch4_window12_384.ms_in22k
5,swin_base_patch4_window12_384.ms_in22k_ft_in1k
6,swin_large_patch4_window7_224.ms_in22k
7,swin_large_patch4_window7_224.ms_in22k_ft_in1k
8,swin_large_patch4_window12_384.ms_in22k
9,swin_large_patch4_window12_384.ms_in22k_ft_in1k


#### Инициализация моделей

Следующий cell пока не используется в связи с ограниченностью ресурсов. Изначально планировалось вести список models (моделей), а в них разные dataloaders(подгрузчики datasets). Однако такое решение невозможно. Поэтому далее происходит инициализация только 1 объекта, а не в цикле

In [ ]:
# num of models in list
models_num = 1
models_names_lst = timm.list_models('swin*', pretrained=True)
models_names_lst = models_names_lst[:models_num]
print(models_names_lst)

['swin_base_patch4_window7_224.ms_in1k']


In [ ]:
models = [Model(model_name=model_name, device=device, dtype=dtype)
             for model_name in models_names_lst]

## Подготовка datasets
В этом разделе подгружается определенный dataset. После происходит процесс разбивки опреленного dataset на части train, validate и test. Эти три части необходимы для создания соответствующих loaders, которые нужны при train и test


#### Загрузка datasets

Всего будет использовано три dataset'а. Список используемых datasets ниже:
1. CIFAR-100:
   - Размер: Датасет CIFAR-100 содержит 60000 цветных изображений размером 32x32 пикселя, разделенных на 50000 изображений для обучения и 10000 для тестирования.
   - Количество классов: CIFAR-100 состоит из 100 различных классов, каждый из которых представляет собой категорию объектов или сцен.
   - Распределение классов: Каждый класс в CIFAR-100 содержит ровно 600 изображений, что означает равномерное распределение классов.
   - Тип данных: Датасет состоит из цветных изображений, относящихся к различным категориям объектов и сцен.
   - Аннотации или разметка: Каждое изображение в CIFAR-100 имеет соответствующую метку класса, позволяющую классифицировать изображения.

2. zh-plus/tiny-imagenet:
   - Размер: Датасет "zh-plus/tiny-imagenet" имеет размер около 2.9 ГБ и содержит более 100 тыс. изображений разного размера.
   - Количество классов: В наборе данных "zh-plus/tiny-imagenet" содержится 200 классов, что представляет собой набор категорий различных объектов.
   - Распределение классов: Каждый класс в "zh-plus/tiny-imagenet" содержит около 500 изображений, а каждая категория имеет свою уникальную подпапку в датасете.
   - Тип данных: Датасет "zh-plus/tiny-imagenet" состоит из цветных изображений различного размера.
   - Аннотации или разметка: Каждое изображение в датасете сопровождается меткой класса (название категории), что облегчает его классификацию или обработку.
   
3. Food-101:
   - Размер: Датасет Food-101 содержит 101000 изображений различных блюд, общим объемом более 5 ГБ.
   - Количество классов: В Food-101 имеется 101 класс, в каждом из которых представлены различные типы пищевых продуктов и блюд.
   - Распределение классов: Каждый класс в Food-101 содержит примерно 1000 изображений, обеспечивая сбалансированное распределение классов.
   - Тип данных: Датасет состоит из цветных изображений блюд и пищевых продуктов.
   - Аннотации или разметка: В Food-101 каждое изображение со
     ровождается меткой класса (тип блюда или продукта), что облегчает

     *Могут попадаться "случайные изображения" в grayscale!* его классификацию.

In [ ]:
# datasets_names_lst = ['cifar100', 'zh-plus/tiny-imagenet', 'food101']
dataset_name = 'food101'
print(f"HuggingFace Dataset: {dataset_name}")
cache_dir = Path(f'{dataset_dir}/{dataset_name}')
print(f"Dataset Path: {cache_dir}")
num_workers = multiprocessing.cpu_count()
# Load the dataset from Hugging Face Hub
main_dataset = load_dataset(dataset_name,
                           cache_dir=cache_dir,
                           num_proc=num_workers)
# get num of classes in dataset
features = main_dataset['train'].features.values()
if (len(features) == 2):
  images, labels = features
else:
  images, labels, _ = features
num_classes = len(labels.names)

print(num_classes, main_dataset)

HuggingFace Dataset: food101
Dataset Path: /content/datasets/food101


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7a5a80c50ca0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1409, in _shutdown_workers
    if not self._shutdown:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_shutdown'


101 DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 75750
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 25250
    })
})


#### Изменение (разбивка) dataset'а

In [ ]:
"""
# reformat cifar100 dataset
dataset = main_dataset
temp = DatasetDict()
train_split, \
  val_split = dataset['train'].train_test_split(test_size=1/10).values()
temp["train"] = train_split
temp["valid"] = val_split
temp["test"] = dataset['test']
main_dataset = temp
print(main_dataset)


# reformat cats_vs_dogs dataset
dataset = datasets[1]
temp = DatasetDict()
train_split, \
  test_split = dataset['train'].train_test_split(test_size=1/6).values()
train_split, \
  val_split = train_split.train_test_split(test_size=1/10).values()
temp["train"] = train_split
temp["valid"] = val_split
temp["test"] = test_split
datasets[1] = temp
"""
# reformat food101 dataset

dataset = main_dataset
temp = DatasetDict()
train_split, \
  val_split = dataset['train'].train_test_split(test_size=1/10).values()
temp["train"] = train_split
temp["valid"] = val_split
temp["test"] = dataset['validation']
main_dataset = temp


#### Код удаления директории
Нужно для того, чтобы удалить папку downloads в dataset'ах

In [ ]:
def delete_files_in_directory(directory: Path):
    if not directory.exists():
        print(f"Directory {directory} does not exist.")
        return

    for item in directory.glob('*'):
        if not item.is_file():
            continue

        try:
            item.unlink()
        except Exception as e:
            print(f"Unable to delete file {item}. Error: {e}")

cache_dir = Path(f'{dataset_dir}/{dataset_name}')
download_directory = Path(os.path.join(cache_dir, "downloads"))
delete_files_in_directory(download_directory)

### Dataset class

Next, we define a custom PyTorch Dataset class that will get used in a DataLoader to create batches. This class fetches a sample from the dataset at a given index and returns the transformed image and its corresponding label index.

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, dataset, classes, tfms):
        self.dataset = dataset
        self.classes = classes
        self.tfms = tfms

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
      sample = self.dataset[idx]
      vals = sample.values()
      if (len(vals) == 2):
        image, label = vals
      else:
        image, label, course_label = vals
      if len(image.mode) != 3:
        image = image.convert("RGB")
      tensor_image = self.tfms(image)
      return tensor_image, label

#### Установка loaders для модели

In [ ]:
def set_loader_to_model(model, dataset, data_loader_params, mark):
  new_dataset = ImageDataset(dataset=dataset,
                             classes=num_classes,
                             tfms=model.tfms)
  data_loader = DataLoader(new_dataset, **data_loader_params, shuffle=True)
  if mark == 'train':
    model.training_data_loader = data_loader
  elif mark == 'test':
    model.testing_data_loader = data_loader
  elif mark == 'valid':
    model.validating_data_loader = data_loader
  else:
    raise Exception("Mark is not correct!")

training_dataset = main_dataset['train']
testing_dataset = main_dataset['test']
validating_dataset = main_dataset['valid']

# dataloader configuration
bs = 64
num_workers = multiprocessing.cpu_count()
# Define parameters for DataLoader
data_loader_params = {
    'batch_size': bs,  # Batch size for data loading
    'num_workers': num_workers,  # Number of subprocesses to use for data loading
    'persistent_workers': True,  # If True, the data loader will not shutdown the worker processes after a dataset has been consumed once. This allows to maintain the worker dataset instances alive.
    'pin_memory': False,  # Set pin_memory to False when using CPU
    'pin_memory_device': device  # Set pin_memory_device to None when using CPU
}
for model in models:
  model.init_model_for_dataset(device, dtype, num_classes)
  model.clear_loaders()
  set_loader_to_model(model, training_dataset, data_loader_params, 'train')
  set_loader_to_model(model, testing_dataset, data_loader_params, 'test')
  set_loader_to_model(model, validating_dataset, data_loader_params, 'valid')

for model in models:
  print(model.testing_data_loader)
  print(model.training_data_loader)
  print(model.validating_data_loader)

((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))


## Testing/Training
Здесть происходит тестирование или тренировка модели под уже определенный ранее dataset

#### Pre(testing/training) seting modeling parameters

In [ ]:
lr = 1e-3
epochs = 3
use_amp = torch.cuda.is_available()
print(use_amp)

def set_modeling_parameters(model, dataloader):
  optimizer = torch.optim.AdamW(model.parameters(), lr=lr, eps=1e-5)
  lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer,
                                                    max_lr=lr,
                                                    total_steps=epochs*len(dataloader))
  metric = MulticlassAccuracy()
  return optimizer, metric, lr_scheduler, device

False


#### Epoch and training loop

In [ ]:
from time import perf_counter

def run_epoch(model, dataloader, optimizer, metric, lr_scheduler, device, scaler, is_training):
    model.train() if is_training else model.eval()
    metric.reset()
    epoch_loss = 0
    progress_bar = tqdm(total=len(dataloader), desc="Train" if is_training else "Eval")
    t0 = perf_counter()
    for batch_id, (inputs, targets) in enumerate(dataloader):
        print(1 if model is not None else 0)
        # inputs, targets = inputs.to(device), targets.to(device)
        with torch.set_grad_enabled(is_training):
            with autocast(device):
              outputs = model(inputs)
              loss = torch.nn.functional.cross_entropy(outputs, targets)
        metric.update(outputs.detach().cpu(), targets.detach().cpu())
        if is_training:
            if scaler is not None:
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                loss.backward()
                optimizer.step()

            optimizer.zero_grad()
            lr_scheduler.step()

        loss_item = loss.item()
        epoch_loss += loss_item
        progress_bar.set_postfix(accuracy=metric.compute().item(),
                                 loss=loss_item,
                                 avg_loss=epoch_loss/(batch_id+1),
                                 lr=lr_scheduler.get_last_lr()[0] if is_training else "")
        progress_bar.update()
        if math.isnan(loss_item) or math.isinf(loss_item):
            break
    t1 = perf_counter()
    progress_bar.close()
    if is_training:
      return epoch_loss / (batch_id + 1)
    else:
      return epoch_loss / (batch_id + 1), {'accuracy': metric.compute().item(),
                                           's/img': (t1 - t0) / (len(dataloader) * bs),
                                           'img/s': (len(dataloader) * bs) / (t1 - t0),
                                           'it/s': (len(dataloader) / (t1 - t0)),
                                           'proc_time': t1 - t0}

In [ ]:
# Main training loop
def train_loop(model, train_dataloader, valid_dataloader, optimizer, metric, lr_scheduler, device, epochs, use_amp, checkpoint_path):
    scaler = GradScaler() if use_amp else None
    best_loss = float('inf')
    for epoch in tqdm(range(epochs), desc="Epochs"):
        train_loss = run_epoch(model, train_dataloader, optimizer, metric, lr_scheduler, device, scaler, is_training=True)
        with torch.no_grad():
            valid_loss = run_epoch(model, valid_dataloader, None, metric, None, device, scaler, is_training=False)
        if valid_loss < best_loss:
            best_loss = valid_loss
            metric_value = metric.compute().item()
            torch.save(model.state_dict(), checkpoint_path)
            training_metadata = {
                'epoch': epoch,
                'train_loss': train_loss,
                'valid_loss': valid_loss,
                'metric_value': metric_value,
                'learning_rate': lr_scheduler.get_last_lr()[0],
            }
            with open(Path(checkpoint_path.parent/'training_metadata.json'), 'w') as f:
                json.dump(training_metadata, f)
        if any(math.isnan(loss) or math.isinf(loss) for loss in [train_loss, valid_loss]):
            print(f"Loss is NaN or infinite at epoch {epoch}. Stopping training.")
            break
    if use_amp:
        torch.cuda.empty_cache()

#### Training

In [ ]:
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
checkpoint_dir = Path(project_dir/f"{timestamp}")
checkpoint_dir.mkdir(parents=True, exist_ok=True)
metrics = []
for model in models:
  checkpoint_path = checkpoint_dir/f"{model.model_name}--cifar100.pth"
  parameters = set_modeling_parameters(model.model, model.training_data_loader)
  train_loop(model.model, model.training_data_loader,
             model.validating_data_loader,
             parameters[0],
             parameters[1],
             parameters[2],
             parameters[3],
             epochs,
             use_amp,
             checkpoint_path)
  print(checkpoint_path)

Epochs:   0%|          | 0/3 [00:00<?, ?it/s]

Train:   0%|          | 0/1066 [00:00<?, ?it/s]

#### Testing

##### Выгрузка модели

In [ ]:
path = f"{models_dir}/{models_weight['food101']}"
# path = f"{project_dir}/{models_weight['food101']}"
# path = f"{project_dir}/{models_weight['zh-plus/tiny-imagenet']}"
print(model.model)
state_dict = torch.load(path, map_location='cpu')
model.model.load_state_dict(state_dict)

SwinTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
  (layers): Sequential(
    (0): SwinTransformerStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): SwinTransformerBlock(
          (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (attn): WindowAttention(
            (qkv): Linear(in_features=128, out_features=384, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=128, out_features=128, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (softmax): Softmax(dim=-1)
          )
          (drop_path1): Identity()
          (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=128, out_features=512, bias=True)
            (act): GELU(approximate='none')
            (

<All keys matched successfully>

##### Testing cell

In [ ]:
metrics = []
for model in models:
    parameters = set_modeling_parameters(model.model, model.testing_data_loader)
    metrics.append((model.model_name, dataset_name,
                  run_epoch(model.model, model.testing_data_loader, *parameters, scaler=None, is_training=False)))
print(metrics)

Eval:   0%|          | 0/395 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:597: UserWarning: pin memory device is set and pin_memory flag is not used then device pinned memory won't be usedplease set pin_memory to true, if you need to use the device pin memory
  warnings.warn(warn_msg)


1
1
1
1


In [ ]:
print(len(model.testing_data_loader))

395


#### Download weights

In [ ]:
from google.colab import files
files.download("/content/image-classifier/2023-10-18_12-16-52/swin_base_patch4_window7_224.ms_in1k--cifar100.pth")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>